A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [28]:
# Load libraries
import pandas as pd

# Load data
ds_jobs = pd.read_csv('customer_train.csv')

# Copy the DataFrame for cleaning
ds_jobs_clean = ds_jobs.copy()
 
# Inspect data
display(ds_jobs_clean.head())

student_id      city  city_development_index gender  \
0        8949  city_103                   0.920   Male   
1       29725   city_40                   0.776   Male   
2       11561   city_21                   0.624    NaN   
3       33241  city_115                   0.789    NaN   
4         666  city_162                   0.767   Male   

       relevant_experience enrolled_university education_level  \
0  Has relevant experience       no_enrollment        Graduate   
1   No relevant experience       no_enrollment        Graduate   
2   No relevant experience    Full time course        Graduate   
3   No relevant experience                 NaN        Graduate   
4  Has relevant experience       no_enrollment         Masters   

  major_discipline experience company_size    company_type last_new_job  \
0             STEM        >20          NaN             NaN            1   
1             STEM         15        50-99         Pvt Ltd           >4   
2             STEM          5          NaN             NaN        never   
3  Business Degree         <1          NaN         Pvt Ltd        never   
4             STEM        >20        50-99  Funded Startup            4   

   training_hours  job_change  
0              36           1  
1              47           0  
2              83           0  
3              52           1  
4               8           0

In [29]:
# Check the data types in the file
display(ds_jobs_clean.dtypes)

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object

In [30]:
# Check the unique values in columns with ordered categorical data in order to create a dictionary
display(ds_jobs_clean['relevant_experience'].value_counts())
display(ds_jobs_clean['enrolled_university'].value_counts())
display(ds_jobs_clean['education_level'].value_counts())

relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: count, dtype: int64

enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64

education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: count, dtype: int64

In [31]:
# Create a list of columns with ordered categorical data
ordered_categories = {
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Create a list of columns with nominal categorical data
# nominal_categories = ['city', 'gender', 'major_discipline']

# Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
# Columns containing nominal categorical data must be stored as the category data type.
# Loop through DataFrame columns to change data types
for col in ds_jobs_clean:
    
    # Convert integer columns to int32
    if ds_jobs_clean[col].dtype == 'int':
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('int32')
    
    # Convert float columns to float16
    elif ds_jobs_clean[col].dtype == 'float':
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('float16')
    
    # Convert columns containing ordered categorical data to ordered categories using dict
    elif col in ordered_categories.keys():
        category = pd.CategoricalDtype(ordered_categories[col], ordered=True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)
    
    # Convert remaining columns to standard categories
    else:
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('category')

In [32]:
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]